In [1]:
from transformers import TFAutoModel
import tensorflow as tf
from tensorflow import TensorSpec
import os
os.chdir(r'C:\Users\Rushi.Soni\Documents\topic_modeling\data')

In [2]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [3]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

In [4]:
# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access final activations (alread max-pooled) [1]
# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(x)

In [5]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
model.layers[2].trainable = False

# print out model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [6]:
optimizer = tf.keras.optimizers.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

C:\Users\Rushi.Soni\AppData\Local\Continuum\anaconda3\envs\transformers_dl\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
element_spec = ({'input_ids': TensorSpec(shape=(32, 512), dtype=tf.int32, name=None),
                 'attention_mask': TensorSpec(shape=(32, 512), dtype=tf.int32, name=None)},
                 TensorSpec(shape=(32, 5), dtype=tf.float64, name=None))
 
# load the training and validation sets
train_ds = tf.data.experimental.load('train', element_spec=element_spec)
val_ds = tf.data.experimental.load('vali', element_spec=element_spec)

# view the input format
train_ds.take(1)

<TakeDataset shapes: ({input_ids: (32, 512), attention_mask: (32, 512)}, (32, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [8]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
213/213 [==============================] - 9452s 44s/step - loss: 1.5632 - accuracy: 0.2848 - val_loss: 1.5527 - val_accuracy: 0.2948
Epoch 2/3
213/213 [==============================] - 9442s 44s/step - loss: 1.5411 - accuracy: 0.3000 - val_loss: 1.5403 - val_accuracy: 0.3078
Epoch 3/3
213/213 [==============================] - 43950s 207s/step - loss: 1.5310 - accuracy: 0.3168 - val_loss: 1.5311 - val_accuracy: 0.3137


In [9]:
model.save('sentiment_model')

INFO:tensorflow:Assets written to: sentiment_model\assets


INFO:tensorflow:Assets written to: sentiment_model\assets
C:\Users\Rushi.Soni\AppData\Local\Continuum\anaconda3\envs\transformers_dl\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
